In [2]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as f
from torch.optim import adam

In [13]:
class multimodal_cnn(nn.Module):
    def __init__(
        self,
        samples,
        lookback,
        kernel_size,
        hidden_size
        ):
        
        super.__init__()
        self.samples = samples
        self.lookback = lookback
        self.kernel_size = kernel_size  # expected odd kernel_size
        self.hidden_size = hidden_size
        
        self.cnn1 = nn.Conv3d(1, 1, self.kernel_size, padding=int((self.kernel_size - 1) / 2))
        self.cnn2 = nn.Conv2d(1, 1, self.kernel_size, padding=int((self.kernel_size - 1) / 2))
        self.fusion = nn.Linear(lookback * samples * 6, hidden_size)  # linear or bilinear? not sure yet.
        self.fc = nn.Linear(hidden_size , 3)
    
    def forward(self, physical, time):
        x = f.relu(self.cnn1(physical))
        x = f.max_pool2d(x, kernel_size=self.kernel_size, stride=1, padding=int((self.kernel_size - 1) / 2))
        x = torch.flatten(x, 1)
        
        y = f.relu(self.cnn2(time))
        y = f.max_pool2d(y, kernel_size=self.kernel_size, stride=1, padding=int((self.kernel_size - 1) / 2))
        y = torch.flatten(y, 1)

        combined = torch.cat([x, y], dim=1)
        fused = self.dropout(f.relu(self.fusion(combined)))
        fc = self.fc(fused)
        pred = f.softmax(fc)
        
        return pred

In [11]:
def fit_model():
    x = pd.read_csv('appa1_adjusted.csv')
    x = x[x['sensing_material'] == 'ZnOR-1'].reset_index()
    gt = pd.read_csv('appa1_gt.csv')
    common = pd.merge(x, gt, on='ts')
    x = x[x.ts.isin(common.ts)]
    y = y[y.ts.isin(common.ts)]
    physical = x[['signal_res', 'heater_res', 'heater_volt', 'T', 'RH', 'P']]
    time = x[['age', 'sin_hour', 'cos_hour', 'sin_ordate', 'cos_ordate', 'sin_weekday', 'cos_weekday', 'year']]

    model = multimodal_cnn()
    optimizer = adam(lr=3e-4)
    criterion = nn.MSELoss()
    for epoch in range(1, 11):
        model.train()
        optimizer.zero_grad()
        pred = model(physical, time)
        loss = criterion(pred, gt)
        print(epoch, loss.item())
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            model.eval()
            pred, loss = model()

In [12]:
fit_model()

TypeError: __init__() missing 5 required positional arguments: 'input_size', 'samples', 'lookback', 'kernel_size', and 'hidden_size'